[link text](https://)**Analyzing U.S. COVID-19 Data**

**Imports:**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy.stats as stats
from google.colab import drive
import statsmodels.api as sm
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report, accuracy_score


**Loading the data:**

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data1_path = '/content/drive/Shareddrives/team6/data.csv'

df1 = pd.read_csv(data1_path)


<ipython-input-3-c267239a5a68>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(data1_path)


In [ ]:
df1_cleaned = df1.drop_duplicates()

**PART 3: Hypothesis Testing:**

**Claim: “There is a strong association between probability of death due to COVID-19 and patient demographics”**

  **3.1 Formulate a hypothesis test to assess the validity of this claim given**
the available data:

● State the test you will use and justify your choice.

● Clearly state the hypotheses.

● Conduct the test and report the result.

● Make a conclusion as to the validity of the claim, assume a significance level
of 0.05.

we will use The Chi-Square Test as Chi-Square Test of Independence is suitable for this analysis because it is designed to assess whether there is a significant association between two categorical variables. In this case, the categorical variables are the combined demographics and the death outcome.

Null Hypothesis (H₀): There is no association between the probability of death due to COVID-19 and patient demographics

Alternative Hypothesis (H₁): There is an association between the probability of death due to COVID-19 and patient demographics

In [ ]:
df = df1_cleaned[['age_group', 'sex', 'race', 'death_yn']].copy()
df = df[~df['age_group'].isin(['Missing', 'Unknown'])]
df = df[~df['sex'].isin(['Missing', 'Unknown'])]
df = df[~df['race'].isin(['Missing', 'Unknown'])]
df = df[~df['death_yn'].isin(['Missing', 'Unknown'])]
df.dropna(inplace=True)
df['demographics'] = df[['age_group', 'sex', 'race']].astype(str).agg('_'.join, axis=1)
df

,age_group,sex,race,death_yn,demographics
4,65+ years,Female,White,No,65+ years_Female_White
10,18 to 49 years,Female,Black,No,18 to 49 years_Female_Black
12,18 to 49 years,Female,White,No,18 to 49 years_Female_White
13,50 to 64 years,Female,White,No,50 to 64 years_Female_White
16,0 - 17 years,Male,White,No,0 - 17 years_Male_White
...,...,...,...,...,...
19020868,65+ years,Female,White,No,65+ years_Female_White
19020903,18 to 49 years,Female,White,No,18 to 49 years_Female_White
19020907,50 to 64 years,Male,White,No,50 to 64 years_Male_White
19020912,18 to 49 years,Male,White,No,18 to 49 years_Male_White


In [ ]:
df_copy = df[['demographics', 'death_yn']].copy()

# Create a contingency table
contingency_table = pd.crosstab(df_copy['demographics'], df_copy['death_yn'])

# Perform the chi-square test
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Print the results
print("Chi-square statistic:", chi2)
print("p-value:", p)


Chi-square statistic: 126131.64445956664
p-value: 0.0


we will Reject the null hypothesis (H₀) as the p value is less than the significance level : There is an association between the probability of death due to COVID-19 and patient demographics.

In [ ]:
contingency_table

death_yn,No,Yes
demographics,,
0 - 17 years_Female_American Indian/Alaska Native,1013,0
0 - 17 years_Female_Asian,4186,0
0 - 17 years_Female_Black,20905,0
0 - 17 years_Female_Multiple/Other,2775,0
0 - 17 years_Female_Native Hawaiian/Other Pacific Islander,43,0
0 - 17 years_Female_White,89987,0
0 - 17 years_Male_American Indian/Alaska Native,1048,0
0 - 17 years_Male_Asian,4588,0
0 - 17 years_Male_Black,21044,0




```

```

 **3.2 Come up with your own claim from the available data and conduct
a hypothesis test for it following in the same steps.**

claim: Investigate whether patients with exposure are more likely to die.





**● State the test you will use and justify your choice:**

we will use z-test for proportions. This test is chosen because it assesses whether there is a significant difference in proportions between two independent groups (patients with exposure and patients without exposure). In this case, we are interested in comparing the proportion of deaths (a binary outcome) between these two groups. The z-test for proportions allows us to determine if the observed difference in proportions is statistically significant, providing valuable insights into the relationship between exposure and mortality risk in the patient population.

**● Clearly state the hypotheses.**

Null Hypothesis (H_0):

There is no significant association between exposure and the proportion of patients that death.

Alternative Hypothesis (H_1):

There is a significant association between exposure and the proportion of patients that death.

**● Conduct the test and report the result.**

In [ ]:
data = df1_cleaned [df1_cleaned['death_yn'].isin(['Yes', 'No'])]
df= data[['exposure_yn', 'death_yn']].copy()


In [ ]:
# Filter patients based on exposure status
patients_with_exposure = df[(df['exposure_yn'] == 'Yes') | (df['exposure_yn'] == 'Unknown')]
patients_without_exposure = df[(df['exposure_yn'] == 'Missing')]

# Further filter patients based on death status
patients_with_exposure_and_death = patients_with_exposure[patients_with_exposure['death_yn'] == 'Yes']
patients_without_exposure_and_death = patients_without_exposure[patients_without_exposure['death_yn'] == 'Yes']

# Calculate the number of patients in each group
num_with_exposure = len(patients_with_exposure)
num_without_exposure = len(patients_without_exposure)

# Calculate the number of deaths in each group
num_deaths_with_exposure = len(patients_with_exposure_and_death)
num_deaths_without_exposure = len(patients_without_exposure_and_death)

# Perform the z-test for proportions
z_score, p_value = sm.stats.proportions_ztest(
    [num_deaths_with_exposure, num_deaths_without_exposure],
    [num_with_exposure, num_without_exposure],
    alternative='larger'
)

# Output the results
z_score, p_value


(-70.24192843956261, 1.0)

 **make a conclusion as to the validity of the claim, assume a significance level
of 0.05.**

as shown p value is greater than the significance level "0.05" so we Fail to reject the null hypothesis: There is no significant association between exposure and the proportion of patients that death.

**Bonus:**

 **Train a machine/deep learning classifier to predict the likelihood of death due to
COVID-19 using any/all of the relevant attributes in the COVID case surveillance
dataset.**

In [ ]:
df = df1_cleaned[['age_group', 'sex', 'race', 'death_yn']].copy()
df = df[~df['age_group'].isin(['Missing', 'Unknown'])]
df = df[~df['sex'].isin(['Missing', 'Unknown'])]
df = df[~df['race'].isin(['Missing', 'Unknown'])]
df = df[~df['death_yn'].isin(['Missing', 'Unknown'])]
df.dropna(inplace=True)

# One-Hot Encoding for categorical features
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_features = encoder.fit_transform(df[['age_group', 'sex', 'race']])

# Label Encoding for target variable
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['death_yn'])

# Combine encoded features with labels
X = encoded_features
y = encoded_labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print evaluation metrics
print(classification_report(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/50
57020/57020 [==============================] - 195s 3ms/step - loss: -10.4144 - accuracy: 0.5846 - val_loss: -55.3508 - val_accuracy: 0.6032
Epoch 2/50
57020/57020 [==============================] - 210s 4ms/step - loss: -188.4778 - accuracy: 0.5951 - val_loss: -489.1005 - val_accuracy: 0.5982
Epoch 3/50
57020/57020 [==============================] - 201s 4ms/step - loss: -861.7496 - accuracy: 0.5663 - val_loss: -1697.5332 - val_accuracy: 0.6709
Epoch 4/50
57020/57020 [==============================] - 206s 4ms/step - loss: -2368.7974 - accuracy: 0.6488 - val_loss: -4096.1187 - val_accuracy: 0.5915
Epoch 5/50
57020/57020 [==============================] - 194s 3ms/step - loss: -5058.5264 - accuracy: 0.6490 - val_loss: -8100.5522 - val_accuracy: 0.7176
Epoch 6/50
57020/57020 [==============================] - 188s 3ms/step - loss: -9267.9121 - accuracy: 0.6490 - val_loss: -14066.2119 - val_accuracy: 0.6944
Epoch 7/50
57020/57020 [==============================] - 205s 4ms/ste

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.79      0.91      0.85    349394
           1       0.01      0.07      0.02      9516
           2       0.00      0.00      0.00     97248

    accuracy                           0.70    456158
   macro avg       0.27      0.33      0.29    456158
weighted avg       0.61      0.70      0.65    456158

Accuracy: 0.6994221300514295


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
